In [1]:
"""
Ingredient Full Cleaner Pipeline
--------------------------------
1️⃣ recipe_by_type.csv 파일을 읽어
2️⃣ ingredient_full 컬럼의 문자열 리스트를 {'재료명': '양/단위'} 딕셔너리로 변환
3️⃣ IngredientETL의 네이밍 정제 규칙을 적용 (양념류 제거 ❌)
4️⃣ 정제된 딕셔너리를 다시 ingredient_full에 업데이트 후 CSV로 저장
"""

import pandas as pd
import re
import ast
import logging
from typing import List, Set


# --------------------------
# 1. IngredientETL 클래스
# --------------------------
class IngredientETL:
    def __init__(self):
        # 고기류 표준화
        self.meat_patterns = {
            r'소고기.*양지|양지머리|소고기양지': '소고기_양지',
            r'소고기.*국거리': '소고기_국거리',
            r'소고기.*다짐육': '소고기_다짐육',
            r'쇠고기': '소고기_양지',
            r'돼지고기.*다짐육': '돼지고기_다짐육',
            r'돼지고기.*삼겹살': '돼지고기_삼겹살',
            r'닭가슴살|닭.*가슴살': '닭고기_가슴살',
            r'닭다리|닭.*다리': '닭고기_다리',
            r'베이컨': '베이컨'
        }

        # 불필요한 설명 제거
        self.description_patterns = [
            (r'계란.*흰자|달걀.*흰자', '계란'),
            (r'계란.*노른자|달걀.*노른자', '계란'),
            (r'(빨강|빨간|노랑|노란|초록|청|홍|황)[ ]*파프리카', '파프리카'),
            (r'(빨강|빨간|노랑|노란|초록|청|홍|황)[ ]*피망', '피망'),
            (r'다진|썬|깐|갈은|가루|곱게|굵게|큼직하게|먹기.*좋게|한입.*크기', ''),
            (r'\([^)]*\)|\[[^\]]*\]', ''),  # 괄호 내용 제거
        ]

    def remove_description(self, ingredient: str) -> str:
        """ 불필요한 단어 제거 """
        result = ingredient
        for pattern, replacement in self.description_patterns:
            result = re.sub(pattern, replacement, result)
        return ' '.join(result.split()).strip()

    def normalize_meat(self, ingredient: str) -> str:
        """ 고기류 표준화 """
        for pattern, standard in self.meat_patterns.items():
            if re.search(pattern, ingredient):
                return standard
        return ingredient.strip()


# --------------------------
# 2. 재료명/단위 분리 함수
# --------------------------
def make_ingredient_dict(ingredient_list):
    """
    재료 리스트를 {'재료명': '양/단위'} 형태로 변환
    """
    ingredient_dict = {}
    amount_keywords = [
        '약간', '적당히', '조금', '큼직하게', '솔솔', '톡톡', '적당량', '한줌',
        '보통사이즈', '작은거', '큰캔', '선택사항', '생략가능', '크게', '깍아서',
        '중간크기', '중간사이즈', '또는', '손가락길이', '손가락 길이', '정도', '소량',
        '듬뿍', '넉넉히', '취향껏', '원하는만큼', '기호에맞게', '필요한만큼'
    ]

    amount_pattern = re.compile(
        r'('
        r'(?:\d[\d\/\.\~]*\s*[가-힣A-Za-z]+)'
        + '|' + '|'.join(amount_keywords) +
        r')'
    )

    for item in ingredient_list:
        if not item or not isinstance(item, str):
            continue

        # 연속된 공백 기준 split
        parts = re.split(r'\s{2,}', item.strip())
        if len(parts) == 2:
            name, raw_amount = parts[0].strip(), parts[1].strip()
        else:
            # 숫자 또는 키워드로 구분
            parts = re.split(
                r'\s*(?=\d|' + '|'.join(amount_keywords) + r')', item
            )
            name = parts[0].strip()
            matches = amount_pattern.findall(item)
            raw_amount = ' '.join(matches).strip() if matches else ""

        # 정제
        name = re.sub(r'[^가-힣A-Za-z0-9\s]', '', name).strip()
        raw_amount = raw_amount.replace('  ', ' ').strip()

        if name:
            ingredient_dict[name] = raw_amount or None

    return ingredient_dict


# --------------------------
# 3. 메인 파이프라인 실행
# --------------------------
def run_pipeline(input_csv="recipe_by_type.csv", output_csv="recipe_by_type_cleaned.csv"):
    etl = IngredientETL()
    df = pd.read_csv(input_csv)
    processed_ingredients = []

    for idx, row in df.iterrows():
        try:
            # 1️⃣ 문자열 리스트 → 리스트 변환
            ingredient_list = ast.literal_eval(row['ingredient_full'])

            # 2️⃣ 재료명/단위 분리
            ingredient_dict = make_ingredient_dict(ingredient_list)

            # 3️⃣ 네이밍 정제
            cleaned_dict = {}
            for name, amount in ingredient_dict.items():
                normalized = etl.normalize_meat(etl.remove_description(name))
                if normalized:
                    cleaned_dict[normalized] = amount

            processed_ingredients.append(cleaned_dict)

        except Exception as e:
            print(f"❌ {row.get('recipe_nm_ko', 'Unknown')} 처리 오류: {e}")
            processed_ingredients.append(None)

    # 4️⃣ 정제 결과 덮어쓰기 후 저장
    df['ingredient_full'] = processed_ingredients
    # ✅ 저장 전에 전체 ingredient_full 컬럼 확인
    print("\n🔍 [전처리 결과 전체 확인 - ingredient_full 컬럼]\n")
    df[['ingredient_full']].to_csv("ingredient_full_only.csv", index=False, encoding="utf-8-sig")


    #df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    #print(f"✅ CSV 저장 완료 → {output_csv}")


# --------------------------
# 4. 실행
# --------------------------
if __name__ == "__main__":
    run_pipeline()



🔍 [전처리 결과 전체 확인 - ingredient_full 컬럼]

